In [ ]:
!pip install wandb tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=08144bf40e3147b745a601009efb72dc8eb2981fb1fae9a1d2415e03f13b73f0
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras as ks
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, mean_poisson_deviance, 
    brier_score_loss, roc_auc_score, roc_curve, RocCurveDisplay
)

import pickle
from datetime import datetime

import wandb
wandb.login(relogin = True)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Prep Data Once

In [ ]:
def filter_nested_array(x, keep):
  result = [[y[key] for key in keep] for y in x]

  return np.array(result, dtype = 'object')

def prepare_row(row):
  # Figure out Claims
  claims_keep_key = ['bi_ind', 'coll_ind', 'comp_ind', 'ers_ind', 'mpc_ind', 'pd_ind', 'ubi_ind',
                      'veh_had_bi_cov_ind', 'veh_had_coll_cov_ind', 'veh_had_comp_cov_ind', 'veh_had_ers_cov_ind', 
                     'veh_had_mpc_cov_ind', 'veh_had_pd_cov_ind', 'veh_had_ubi_cov_ind']
                      
  other_claims = filter_nested_array(row['other_claims'], claims_keep_key)
  other_claim_cnt = len(other_claims)
  if other_claim_cnt > 0:
    other_claims = np.append(other_claims, np.zeros([len(other_claims),1]), 1)

  veh_claims = filter_nested_array(row['vehicle_claims'], claims_keep_key)
  claim_cnt = len(veh_claims)
  if claim_cnt > 0:
    veh_claims = np.append(veh_claims, np.ones([len(veh_claims),1]), 1)

  if claim_cnt + other_claim_cnt == 0:
      all_claims = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
  elif claim_cnt == 0:
    all_claims = other_claims
  elif other_claim_cnt == 0:
    all_claims = veh_claims
  else:
    all_claims = np.append(veh_claims, other_claims, axis = 0)

  all_claims = tf.ragged.constant([all_claims.astype('float16')], ragged_rank = 1, inner_shape = (15,))

  # Figure Out Drivers
  drivers = filter_nested_array(row['driver_info'], ['driver_age', 'driver_gender', 'driver_tenure'])
  drivers[:, 0] = (drivers[:, 0] - 50)/50
  drivers[:, 1] = np.where(drivers[:, 1] == 'm', 1, 0)
  drivers[:, 2] = (drivers[:, 2] - 10)/10
  drivers = tf.ragged.constant([drivers.astype('float16')], ragged_rank = 1, inner_shape = (3,))

  # Figure out Vehicles
  vehicles = filter_nested_array(row['household_vehicles_info'], ['this_vehicle_ind', 'vehicle_age', 'vehicle_type', 'vehicle_years_owned'])
  vehicles[:, 1] = (vehicles[:, 1] - 15)/15
  vehicles[:, 3] = (vehicles[:, 3] - 15)/15

  veh_type = vehicles[:, 2]
  vehicles[:, 2] = np.where(veh_type == 'van', 1, 0) + np.where(veh_type == 'sedan', 2, 0) + np.where(veh_type == 'sports car', 3, 0) + np.where(veh_type == 'suv', 4, 0)
  vehicles = tf.ragged.constant([vehicles.astype('float16')], ragged_rank = 1, inner_shape = (4,))

  other = [(row['credit_score'] - 600)/500,
           (row['garaging_location'] == 'country') * 1 + (row['garaging_location'] == 'downtown') * 2, 
           (row['household_tenure'] - 15)/10,
           (row['multiline_houses']/2),
           row['multiline_rental'],
           row['multiline_personal_article_policy'],
           row['multiline_personal_liability_umbrella'],
           (row['vehicle_count']-3)/3,
           (row['annual_mileage'] - 10000)/10000,
           (row['vehicle_age'] - 15)/15,
           np.where(row['vehicle_type'] == 'van', 1, 0) + np.where(row['vehicle_type'] == 'sedan', 2, 0) + np.where(row['vehicle_type'] == 'sports car', 3, 0) + np.where(row['vehicle_type'] == 'suv', 4, 0),
           (row['vehicle_years_owned'] - 10)/15,
           (row['max_driver_age'] - 45)/45,
           (row['min_driver_age'] - 45)/45,
           (row['mean_driver_age'] - 45)/45,
           (row['min_driver_tenure'] - 30)/30,
           (row['youthful_driver_count'])/4,
           row['driver_count']/5,
           row['coverage_bi'],
           row['coverage_coll'],
           row['coverage_comp'],
           row['coverage_ers'],
           row['coverage_mpc'],
           row['coverage_pd'],
           row['coverage_ubi']      
           ]
  other = tf.constant(value = np.array(other, dtype = 'float16'))

  target = [
      row['vehicle_claim_cnt_pd_0'],
      row['vehicle_claim_cnt_coll_0'],
      row['vehicle_claim_cnt_bi_0'],
      row['vehicle_claim_cnt_mpc_0']
  ]

  target = tf.constant(value = np.array(target, dtype = 'float16'))

  result = {
      'driver_info': drivers,
      'vehicle_info': vehicles,
      'claims_info': all_claims,
      'other_data': other,
      'target': target
  }

  return result

ragged = lambda y : tf.concat(y.to_list(), axis = 0)

def prep_one_datas(features):

  x = [
      ragged(features['driver_info']), 
      ragged(features['vehicle_info']), 
      ragged(features['claims_info']), 
      tf.convert_to_tensor(features['other_data'].to_list())
  ]

  y = tf.convert_to_tensor(features['target'].to_list())

  return x,y

In [ ]:
with wandb.init(
      project="claims_modeling",
      group = 'Data Prep',
      name = f'Data Prep for NN - {datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}',
      notes="Prep for NN Training, No Model Build!",
      tags=["data"]) as run:
    datas = run.use_artifact('msds_498_claims_modeling/claims_modeling/sythetic_data:v5')
    directory = datas.download(root = 'datasets')

    train_features = pd.read_parquet('datasets/split=train') \
      .apply(prepare_row, axis = 1, result_type = 'expand')

    test_features = pd.read_parquet('datasets/split=test')\
      .apply(prepare_row, axis = 1, result_type = 'expand')

    val_features = pd.read_parquet('datasets/split=validation')\
      .apply(prepare_row, axis = 1, result_type = 'expand')

    train_x, train_y = prep_one_datas(train_features)
    test_x, test_y = prep_one_datas(test_features)
    val_x, val_y = prep_one_datas(val_features)

wandb: Currently logged in as: tylerrosacker2022 (msds_498_claims_modeling). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact sythetic_data:v5, 153.76MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:7.5


# Model Train Code

In [ ]:
def define_loss(weights):
  def weighted_loss(y_true, y_pred):
    fun = tf.keras.losses.Poisson()
    #fun = tf.keras.losses.MeanSquaredError()
    loss = 0

    loss += weights[0] * fun(y_true[:, 0], y_pred[:, 0])
    loss += weights[1] * fun(y_true[:, 1], y_pred[:, 1])
    loss += weights[2] * fun(y_true[:, 2], y_pred[:, 2])
    loss += weights[3] * fun(y_true[:, 3], y_pred[:, 3])

    return loss
  
  return weighted_loss

def build_basic_ragged_layers(name, input_node, width, dropout, agged = True):
  dense_1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(width, 
                                        name = f"{name}_Info_Dense",
                                        activation = tfa.activations.mish,
                                        kernel_initializer='lecun_normal'
                                        ))(input_node)
  normed = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(dense_1)  
  drop = tf.keras.layers.TimeDistributed(tf.keras.layers.AlphaDropout(dropout))(normed)
  
  # dense_2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(width, 
  #                                       name = f"{name}_Info_Dense_2",
  #                                       activation = tfa.activations.mish,
  #                                       kernel_initializer='lecun_normal'
  #                                       ))(drop)
  # normed_2 = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(dense_2)
  # drop_2 = tf.keras.layers.TimeDistributed(tf.keras.layers.AlphaDropout(dropout))(normed_2)

  # combined = tf.keras.layers.Add()([drop, drop_2])

  if agged:
    agged_sum = tf.math.reduce_sum(drop, 1)
    agged_max = tf.math.reduce_max(drop, 1)
    return tf.keras.layers.Concatenate()([agged_sum, agged_max])
  else:
    return drop

def build_basic_combined_layers(name, input_node, width, dropout):
  dense = tf.keras.layers.Dense(width, 
                                  name = f"Res_Layer_{name}",
                                  activation = tfa.activations.mish,
                                  kernel_initializer='lecun_normal'
                                  )(input_node)
  #leaky =  tf.keras.layers.LeakyReLU(alpha=leakiness)(dense)
  drop = tf.keras.layers.AlphaDropout(dropout)(dense)
  norm = tf.keras.layers.BatchNormalization()(drop)

  return norm


def build_model(run):
  # driver_info
  driver_input = ks.Input(shape = (None, 3), name = 'driver_info')
  driver_agged = build_basic_ragged_layers('driver', driver_input, 
                                           width = run.config['driver_dense'],
                                           dropout = run.config['dropout'],
                                           agged = False)

  # vehicle_info
  vehicle_input = ks.Input(shape = (None, 4), name = 'vehicle_info')
  vehicle_agged = build_basic_ragged_layers('vehicle', vehicle_input, 
                                           width = run.config['veh_dense'],
                                           dropout = run.config['dropout'],
                                           agged = False)
  
  driver_vehicle_cross = tf.keras.layers.MultiHeadAttention(num_heads=run.config['attention_heads'], 
                                                            key_dim = run.config['attention_dims'], 
                                                            output_shape = run.config['attention_output_dims']
                                                            )(vehicle_agged, driver_agged)
  attent_normed = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(driver_vehicle_cross)  
  attent_drop = tf.keras.layers.TimeDistributed(tf.keras.layers.AlphaDropout(run.config['dropout']))(attent_normed)
  agged_att_sum = tf.math.reduce_sum(attent_drop, 1)
  agged_att_max = tf.math.reduce_max(attent_drop, 1)

  # claims_info
  claims_input = ks.Input(shape = (None, 15), name = 'claims_info')
  claims_agged = build_basic_ragged_layers('claims', claims_input, 
                                           width = run.config['claim_dense'],
                                        dropout = run.config['dropout'])

  # other_data
  other_input = ks.Input(shape = (25,), name = 'other_data')
  dense_other = build_basic_combined_layers(name = 'other', 
                                        input_node = other_input,
                                        width = run.config['dense_other_block_width'],
                                        dropout = run.config['dropout']
                                        )
  target_other = tf.keras.layers.Dense(4, 
                                      bias_initializer = tf.keras.initializers.Constant(value=run.config['initial_bias']),
                                      activation=tf.keras.activations.exponential, 
                                      name = 'target_other')(dense_other)

  #driver_agged, vehicle_agged, 

  combined = tf.keras.layers.Concatenate()([claims_agged, agged_att_sum, agged_att_max])
  combined_norm = tf.keras.layers.BatchNormalization()(combined)


  dense_1 = build_basic_combined_layers(name = '1', 
                                        input_node = combined_norm,
                                        width = run.config['dense_res_block_width'],
                                        dropout = run.config['dropout']
                                        )
  
  # dense_2 = build_basic_combined_layers(name = '2', 
  #                                       input_node = dense_1,
  #                                       width = run.config['dense_res_block_width'],
  #                                       dropout = run.config['dropout']
  #                                       )

  # res_layer = tf.keras.layers.Add(name = "Combined_Res_Result")([dense_1, dense_2])
  final_features = tf.keras.layers.Concatenate()([dense_1, combined_norm])

  target_ragged = tf.keras.layers.Dense(4, 
                                      bias_initializer = tf.keras.initializers.Constant(value=0),
                                      activation=tf.keras.activations.exponential, 
                                      name = 'target')(final_features)

  final_prediction = tf.keras.layers.Multiply(name = "Final_Prediction")([target_ragged, target_other])

  model = ks.Model(inputs = [driver_input, vehicle_input, claims_input, other_input],
                outputs = [final_prediction])

  model.compile(
                optimizer=tf.keras.optimizers.experimental.AdamW(
                    learning_rate = run.config['learning_rate'],
                    weight_decay = run.config['weight_decay'], 
                    global_clipnorm=5.0,
                    clipvalue=1,
                    amsgrad=True
                ), 
                loss = define_loss(run.config['loss_weight']))
  
  simple_model = ks.Model(inputs = [other_input],
                outputs = [target_other])

  simple_model.compile(
                optimizer=tf.keras.optimizers.experimental.AdamW(
                    learning_rate = run.config['learning_rate'],
                    weight_decay = run.config['weight_decay'], 
                    global_clipnorm=5.0,
                    clipvalue=1,
                    amsgrad=True
                ), 
                loss = define_loss(run.config['loss_weight']))
  
  return model, simple_model

In [ ]:
def log_stats(dataset_name, prediction, truth):
  prediction = prediction[:, 0]
  truth = truth[:, 0]
  
  prediction = np.clip(prediction, a_min = 0.001, a_max = np.inf)  
  predicted_p_gt_0 = np.clip(1 - np.exp(-prediction), a_min = 0, a_max = 1)  
  truth_capped = np.clip(truth, a_min = 0, a_max = 1)
  truth_capped = np.clip(truth, a_min = 0, a_max = 1)

  fpr, tpr, _ = roc_curve(truth_capped, predicted_p_gt_0)
  roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
  
  metrics = {
      f"{dataset_name}_prediction_dist": wandb.Histogram(prediction),
      f"{dataset_name}_mse": mean_squared_error(truth, prediction), 
      f"{dataset_name}_mae": mean_absolute_error(truth, prediction),
      f"{dataset_name}_mean_poisson_deviance": mean_poisson_deviance(truth, prediction),
      f"{dataset_name}_brier_loss": brier_score_loss(truth_capped, predicted_p_gt_0),
      f"{dataset_name}_auc_score": roc_auc_score(truth_capped, predicted_p_gt_0),
      f"{dataset_name}_roc": roc_display.figure_
    }
  wandb.log(metrics)
  


In [ ]:
def main(config = None):
  with wandb.init(
      project="claims_modeling",
      group = 'NN Template V7 - Pretrained Simple',
      name = f'NN Train - {datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}',
      notes="Mutli-target Architecture NN",
      tags=["nn"],
      save_code = True,
      sync_tensorboard=True,
      config=config) as run:
    datas = run.use_artifact('msds_498_claims_modeling/claims_modeling/sythetic_data:v5')
    model, simple_model = build_model(run)

    tf.keras.utils.plot_model(
      model,
      to_file='model.png',
      show_shapes=True,
      show_layer_names=True,
      show_layer_activations=True,
      show_trainable=True
    )

    artifact = wandb.Artifact(
        name='model_arch_graph', 
        type='image'
        )    

    artifact.add_file(local_path='model.png')
    run.log_artifact(artifact)

    epochs = run.config['epochs'] // 4 + 1

    simple_model.fit(train_x[3], 
              train_y, 
              epochs = epochs, 
              batch_size = run.config['batch_size'], 
              validation_data=(test_x[3], test_y))
    
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=1)

    # Treat as transfer learning or nah
    if run.config['keep_trainable'] == False:
      simple_model.trainable = False
      model.fit(train_x, 
                train_y, 
                epochs = 2 * epochs, 
                batch_size = run.config['batch_size'], 
                validation_data=(test_x, test_y),
                callbacks=[tensorboard_callback])
      
      # Now really burn in!
      simple_model.trainable = True
      #K.set_value(model.optimizer.learning_rate, run.config['learning_rate'] * 0.1)
      model.fit(train_x, 
                train_y, 
                epochs = epochs, 
                batch_size = run.config['batch_size'] * run.config['burnin_multiplier'], 
                validation_data=(test_x, test_y),
                callbacks=[tensorboard_callback])
      
    else:
      model.fit(train_x, 
                train_y, 
                epochs = 3 * epochs, 
                batch_size = run.config['batch_size'], 
                validation_data=(test_x, test_y),
                callbacks=[tensorboard_callback])
      
    
    train_pred = model.predict(train_x, batch_size = 1024)
    test_pred = model.predict(test_x, batch_size = 1024)
    val_pred = model.predict(val_x, batch_size = 1024)

    log_stats('train', train_pred, train_y)
    log_stats('test', test_pred, test_y)
    log_stats('val', val_pred, val_y)

    model.save('model')
    wandb.save('model')

# Grid Search!

In [ ]:
# main(config = {
#         "epochs": 50,
#         "learning_rate": 0.01,
#         "weight_decay": 0.00001,
#         "relu_leakiness": 0.015,
#         "driver_dense": 15,
#         "veh_dense": 20,
#         "claim_dense": 25,
#         "attention_heads": 1,
#         "attention_dims": 10,
#         "attention_output_dims": 10,
#         "dense_res_block_width": 25,
#         "dense_other_block_width": 100,
#         "dropout": 0.05,
#         "batch_size": 1024,
#         "initial_bias": -2.5,
#         "loss_weight": [0.7, 0.1, 0.1, 0.1], # [1, 0, 0, 0] [0.4, 0.2, 0.2, 0.2] [0.55, 0.15, 0.15, 0.15] [0.7, 0.1, 0.1, 0.1] [0.85, 0.05, 0.05, 0.05] [0.75, 0.15, 0.05, 0.05] [0.4, 0.3, 0.1, 0.1] [0.25, 0.25, 0.25, 0.25],
#         "keep_trainable": False,
#         "burnin_multiplier": 16
#   })

wandb.agent(entity = "msds_498_claims_modeling", project = "claims_modeling", sweep_id="xw6922l1", function=main, count = 30)

wandb: Agent Starting Run: j7t6n4pe with config:
wandb: 	attention_dims: 250
wandb: 	attention_heads: 3
wandb: 	attention_output_dims: 25
wandb: 	batch_size: 512
wandb: 	burnin_multiplier: 16
wandb: 	claim_dense: 25
wandb: 	dense_other_block_width: 250
wandb: 	dense_res_block_width: 100
wandb: 	driver_dense: 25
wandb: 	dropout: 0.15
wandb: 	epochs: 50
wandb: 	initial_bias: -2.5
wandb: 	keep_trainable: False
wandb: 	learning_rate: 0.005
wandb: 	loss_weight: [0.5, 0.25, 0.05, 0.05]
wandb: 	relu_leakiness: 0.015
wandb: 	veh_dense: 250
wandb: 	weight_decay: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/13
431/431 [==============================] - 9s 7ms/step - loss: 0.2282 - val_loss: 0.2237
Epoch 2/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2231 - val_loss: 0.2227
Epoch 3/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2217 - val_loss: 0.2217
Epoch 4/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2211 - val_loss: 0.2212
Epoch 5/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2206 - val_loss: 0.2216
Epoch 6/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2203 - val_loss: 0.2207
Epoch 7/13
431/431 [==============================] - 3s 6ms/step - loss: 0.2202 - val_loss: 0.2208
Epoch 8/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2200 - val_loss: 0.2210
Epoch 9/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2199 - val_loss: 0.2208
Epoch 10/13
431/431 [==============================] - 2s 5ms/step - loss: 0.2198 - val_loss: 0.2212

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


431/431 [==============================] - 30s 38ms/step - loss: 0.2265 - val_loss: 0.2228
Epoch 2/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2215 - val_loss: 0.2250
Epoch 3/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2210 - val_loss: 0.2220
Epoch 4/26
431/431 [==============================] - 16s 34ms/step - loss: 0.2207 - val_loss: 0.2221
Epoch 5/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2204 - val_loss: 0.2234
Epoch 6/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2200 - val_loss: 0.2226
Epoch 7/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2200 - val_loss: 0.2224
Epoch 8/26
431/431 [==============================] - 16s 34ms/step - loss: 0.2198 - val_loss: 0.2216
Epoch 9/26
431/431 [==============================] - 15s 33ms/step - loss: 0.2197 - val_loss: 0.2217
Epoch 10/26
431/431 [==============================] - 16s 33ms/step - loss: 0.2196 - val_los

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


...

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Epoch 1/7
27/27 [==============================] - ETA: 0s - loss: 0.2671

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


27/27 [==============================] - 9s 272ms/step - loss: 0.2671 - val_loss: 0.2685
Epoch 2/7
27/27 [==============================] - 8s 268ms/step - loss: 0.2667 - val_loss: 0.2687
Epoch 3/7
27/27 [==============================] - 8s 267ms/step - loss: 0.2667 - val_loss: 0.2686
Epoch 4/7
27/27 [==============================] - 8s 266ms/step - loss: 0.2666 - val_loss: 0.2686
Epoch 5/7
27/27 [==============================] - 8s 270ms/step - loss: 0.2666 - val_loss: 0.2686
Epoch 6/7
27/27 [==============================] - 8s 265ms/step - loss: 0.2666 - val_loss: 0.2686
Epoch 7/7
31/31 [==============================] - 1s 20ms/step


global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██████████████
test_auc_score,▁
test_brier_loss,▁
test_mae,▁
test_mean_poisson_deviance,▁
test_mse,▁
train/epoch_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▂▂▂▃▃▃▄▅▅▅▅▆▆▇▇█▁▁▂▂▃▃▄▅
train_auc_score,▁
train_brier_loss,▁
train_mae,▁


In [ ]:
from google.colab import runtime
runtime.unassign()